In [29]:

farmer_bank_details = FarmerBankDetails(
    farmer_id=3254,
    bank="kvb",
    branch="kalla",
    ifsc_code="kvb3456",
    micr_code="34567",
    account_holder_name="ghjk",
    account_number="234567890",
    created_by=212,
    modified_by=212,
)
farmer_bank_details.save()

ValueError: Cannot assign "212": "FarmerBankDetails.created_by" must be a "User" instance.

In [28]:
s=Auth(id=82)
s

NameError: name 'authendicate' is not defined

In [15]:
# print("original to:%s" % to)
# to_good = add_to_phone_number(to)
# print("91 added to:%s" % to_good)
# import plivo
# client = plivo.RestClient(auth_id="MAZJZINTYZZTQ4MTG0MT", auth_token="NWJkN2Q5MGM2OGI2Njc1MGM3NzMzMmMyZjQyYWIw")
# message_created = client.messages.create(
#     src="919500989012", dst=9944334800, text="message"
)

ValidationError: Insufficient credits

In [24]:
import requests
payload = {'ClientId': 'c9dc2b72-a38c-4e29-9834-17fe1ef6df3f', 'ApiKey' :'0fa9908c-1e67-4ccb-86fc-2363f7f75839', 'SenderID' : 'KULTIV', 'fl':'0', 'gwid':'2', 'sid':'KULTIV'} 

headers = {}
url = 'http://45.127.101.185/Login.aspx'
payload['msg'] = "name"
payload['msisdn'] = "9944334800"
res = requests.post(url, data=payload, headers=headers)
print(res)

<Response [200]>


In [ ]:
farmer_agent_obj =  FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id__in=cluster_id_list)
farmer_agent_obj


# master_farmers_report

In [7]:
import pandas as pd
import datetime
from base64 import b64encode, b64decode




data={
            "season_id" : 4,
            "cluster_id_list" :[2,4,15],
             }

season_id = data['season_id']
cluster_id_list = data['cluster_id_list']

if len(cluster_id_list) == 0:
    cluster_id_list = list(ClusterSeasonMap.objects.filter(season_id=season_id).values_list('cluster_id', flat=True))

if 'demo_farmer' in data:
    #demo farmer, agent, cluster, df
    farmer_agent_obj =  FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id__in=cluster_id_list, farmer__is_demo_farmer=True)
else:
    #  master farmer, agent, cluster, dfZx
    farmer_agent_obj =  FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id__in=cluster_id_list)
    farmer_agent_list = farmer_agent_obj.values_list('farmer_id', 'farmer__first_name', 'farmer__last_name', 'seasonal_farmer_code', 'farmer__aadhaar_number','cluster__name','agentfarmermap__agent__first_name', 'userfarmermap__officer__username', 'farmer__village__name', 'farmer__taluk__name', 'farmer__hobli__name', 'farmer__district__name', 'farmer__state__name', 'season__name',)
farmer_agent_column = ['farmer_id', 'first_name', 'last_name', 'farmer_code', 'aadhaar_number','cluster_name','agent_name', 'superviosr_name', 'village', 'taluk', 'hobli', 'district', 'state', 'season_name']
farmer_agent_df = pd.DataFrame(list(farmer_agent_list), columns=farmer_agent_column)
farmer_agent_df = farmer_agent_df.drop_duplicates('farmer_id')

# farmer sowing df
farmer_sowing_list = list(Sowing.objects.filter(cultivation_phase_id=2, season_id=season_id, farmer_id__in=list(farmer_agent_df['farmer_id'])).values_list('farmer_id', 'farmer__mobile', 'area_calculated_via_geo_fencing'))            
farmer_sowing_column = ['farmer_id', 'mobile',  'geo_calculated_area']
farmer_sowing_df = pd.DataFrame(farmer_sowing_list, columns=farmer_sowing_column)
farmer_sowing_df = farmer_sowing_df.fillna(0)
farmer_sowing_df_geo = farmer_sowing_df.groupby('farmer_id').agg({'mobile':'first','geo_calculated_area':sum}).reset_index()

combain_df = pd.merge(farmer_agent_df, farmer_sowing_df_geo, left_on='farmer_id', right_on='farmer_id', how='outer')
combain_df = combain_df.drop_duplicates('farmer_id')

# cultivation pase
phase_list = list(Sowing.objects.filter(season_id=season_id, farmer_id__in=list(combain_df['farmer_id'])).values_list('farmer_id', 'sowing_date','cultivation_phase_id', 'area', 'water_source__name'))
phase_column = ['farmer_id', 'sowing_date','pase_id', 'area', 'main_crop_water_resource_name']
phase_df = pd.DataFrame(phase_list, columns=phase_column)

phase1_df = phase_df[phase_df['pase_id']==1].drop(columns=['pase_id'])
phase1_df = phase1_df.rename(columns={'area': 'total_nursury_crop_area'})
phase1_df = phase1_df.groupby('farmer_id').agg({'total_nursury_crop_area': sum,'main_crop_water_resource_name': 'first', 'sowing_date': 'first'}).reset_index()

phase2_df = phase_df[phase_df['pase_id']==2].drop(columns=['pase_id', 'main_crop_water_resource_name'])
phase2_df = phase2_df.rename(columns={'area': 'total_transplant_crop_area', 'sowing_date': 'Tp_date'})
phase2_df = phase2_df.groupby('farmer_id').agg({'total_transplant_crop_area': sum,  'Tp_date': 'first'}).reset_index()

cultivation_phase_df = pd.merge(phase1_df, phase2_df, left_on='farmer_id', right_on='farmer_id', how='outer')
cultivation_phase_df = cultivation_phase_df.fillna(0)

combain_df = pd.merge(combain_df, cultivation_phase_df, left_on='farmer_id', right_on='farmer_id', how='left')

# farmer bank df
farmer_bank_list = FarmerBankDetails.objects.filter(farmer_id__in=list(combain_df['farmer_id']), is_primary=True).values_list("farmer_id", "bank", "branch", "ifsc_code", "micr_code", "account_holder_name", "account_number", 'remarks')                
farmer_bank_column = ["farmer_id", "bank", "branch", "ifsc_code", "micr_code", "account_holder_name", "account_number", 'remarks']
farmer_bank_df = pd.DataFrame(farmer_bank_list, columns=farmer_bank_column).fillna(0).drop_duplicates('farmer_id')
combain_df = pd.merge(combain_df, farmer_bank_df, left_on='farmer_id', right_on='farmer_id', how='left')

# farmer distribution sowing
sowing_list = list(Sowing.objects.filter(season_id=season_id, farmer_id__in=list(combain_df['farmer_id'])).values_list('id', flat=True))
if AgentFarmerDistributionSowing.objects.filter(sowing_id__in=sowing_list).exists():
    farmer_distribution_list = AgentFarmerDistributionSowing.objects.filter(sowing_id__in=sowing_list).values_list('sowing__farmer_id', 'input_combo__name', 'no_of_unit')
    farmer_distribution_column = ['farmer_id', 'input_combo', 'number_of_unit']
    farmer_distribution_df = pd.DataFrame(farmer_distribution_list, columns=farmer_distribution_column)
    farmer_distribution_df = farmer_distribution_df.groupby(['farmer_id', 'input_combo']).agg({'number_of_unit':sum}).reset_index()

    #->##convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
    farmer_distribution_df = pd.pivot_table(farmer_distribution_df, index=['farmer_id'] , columns='input_combo', aggfunc=min, fill_value=0)

    #->##convert_pivot_table_to_normal_df
    farmer_distribution_df.columns = farmer_distribution_df.columns.droplevel(0) #remove amount
    farmer_distribution_df.columns.name = None  #remove categories
    farmer_distribution_df = farmer_distribution_df.reset_index() #index to columns
    combain_df = pd.merge(combain_df, farmer_distribution_df, left_on='farmer_id', right_on='farmer_id', how='left').fillna(0)

# harvest 
sowing_id_list = list(Sowing.objects.filter(season_id=season_id, farmer_id__in=list(combain_df['farmer_id']), cultivation_phase_id=2).values_list('id', flat=True))
harvest_list = list(Harvest.objects.filter(sowing_id__in=sowing_id_list).values_list('sowing__farmer_id', 'value'))
harvest_column = ['farmer_id', 'harvest_value']

haevesr_df = pd.DataFrame(harvest_list, columns=harvest_column)
haevesr_df = haevesr_df.groupby('farmer_id').agg({'harvest_value':sum}).reset_index()

combain_df = pd.merge(combain_df, haevesr_df, left_on='farmer_id', right_on='farmer_id', how='left').fillna(0)

combain_df['total_yeild_on_sowing _acre'] = combain_df['harvest_value'] / combain_df['total_transplant_crop_area']

combain_df = combain_df.rename(columns={'farmer_id' : "Farmer Id" , 'first_name' : "First Name" , 'last_name' : "Last Name" , 'farmer_code' : "Farmer Code" , 'cluster_name' : "Cluster Name" ,
   'agent_name' : "Agent Name" , 'superviosr_name' : "Superviosr Name" , 'mobile' : "Mobile" , 'village' : "Village" , 'village2': "village2", 'season_name' : "Season Name" ,
   'geo_calculated_area' : "Geo Calculated Area" , 'total_nursury_crop_area' : "Total Nursury Crop Area" ,
   'main_crop_water_resource_name' : "Main Crop Water Resource Name" , 'sowing_date' : "Sowing Date" ,
   'total_transplant_crop_area' : " Total Transplant Crop Area" , 'Tp_date' : "Tp Date" , 'bank' : "Bank" , 'branch' : "Branch" , 'ifsc_code' : "IFSC Code" ,
   'micr_code' : "Micr Code" , 'account_holder_name' : "Account Holder Name" , 'account_number' : "Account Number" , 'remarks' : "Remarks" ,
   'harvest_value' : "Harvest Value" , 'total_yeild_on_sowing _acre' : "Total Yeild On Sowing Acre" ,})

combain_df = combain_df.fillna(0)
combain_df['Farmer Id'] = combain_df['Farmer Id'].astype(str)
combain_df.index += 1
total = combain_df.sum(numeric_only=True)
total.name = 'Total'
combain_df = combain_df.append(total.transpose())

writer = pd.ExcelWriter(str("static/media/") + "master_farmers_report.xlsx", engine="xlsxwriter")
# creating excel sheet with name
combain_df.to_excel(writer, sheet_name="Sheet1", startrow=1)
# assigning that sheet to obj
workbook = writer.book
worksheet = writer.sheets["Sheet1"]
merge_format = workbook.add_format(
    {
        "bold": 1,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
        "fg_color": "yellow",
    }
)

date = datetime.datetime.now().date()
# Merge 3 cells.
worksheet.merge_range("A1:AK1", "Master Farmer Report - " + str(date), merge_format)

format1 = workbook.add_format({"num_format": "#,##0.00"})

# Set the column width and format.
worksheet.set_column("B:B", 18, format1)
worksheet.set_column(0, 17, 20)

# Add a header format.
header_format = workbook.add_format({"fg_color": "#D7E4BC"})

# Write the column headers with the defined format.
for col_num, value in enumerate(combain_df.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)
writer.save()
data = {}
data['excel_name'] = "master_farmer_report.xlsx"
try:
    image_path = str("static/media/") + "master_farmers_report.xlsx"
    with open(image_path, "rb") as image_file:
        encoded_image = b64encode(image_file.read())
        data["excel"] = encoded_image
except Exception as err:
    print(err)
print("haha")

haha


In [21]:
farmer_agent_obj =  FarmerClusterSeasonMap.objects.filter(season_id=4, cluster_id__in=cluster_id_list,farmer__id=6937).values_list('farmer_id', 'farmer__first_name', 'farmer__last_name', 'farmer__code', 'farmer__aadhaar_number','cluster__name','agentfarmermap__agent__first_name', 'userfarmermap__officer__username', 'farmer__village__name', 'farmer__taluk__name')
farmer_agent_obj

<QuerySet [(6937, 'Kumara', 'Kuregowda', '21BMC027', '658099429212', 'Bargi', 'Maheshappa', 'madappa', 'Mukthi  colony', 'Gundlupet')]>

In [ ]:
agent_farmer_list = list(AgentFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame["farmer_code"], farmer__season_id=season_id).values_list("farmer__farmer__code", "agent__first_name"))


In [12]:
sowing_obj = Sowing(
                        farmer_id=11111,
                        crop_id=1,
                        sowing_date='2022-08-15',
                        area=0.5,
                        season_id=4,
                        state_id=1,
                        district_id=36,
                        taluk_id=169,
                        hobli_id=42,
                        village_id=1025,
                        cultivation_phase_id=2,
                        soil_type_id=1,
                        irrigation_method_id=1,
                        water_type_id=None,
                        water_source_id=2,
                    )
sowing_obj.save()

# Non_supplied_farmers

In [16]:
import pandas as pd
import datetime
from base64 import b64encode, b64decode

document = {}
season_id = 4
data_dict_from_interface={"sesson_id":4}

print('data_dict_from_interface')
print('idzzz:')
user_type_id = 6
print(user_type_id)
user_id = 220
print(user_id)
if user_type_id == 5:
    subordinate_user_ids = list(AgentSupervisorSeasonMap.objects.filter(supervisor_id=220, season_id=season_id).values_list('agent_id', flat=True))
else:
    if user_type_id == 3:
        if UserHierarchyMap.objects.filter(superior_id=user_id, season_id=season_id).exists():
            subordinates = UserHierarchyMap.objects.filter(superior_id=user_id, season_id=season_id)
            subordinate_user_ids = list(subordinates.values_list("subordinate", flat=True))
            print("one")
            print(subordinate_user_ids)
            subordinate_user_ids = list(AgentSupervisorSeasonMap.objects.filter(supervisor_id__in=subordinate_user_ids, season_id=season_id).values_list('agent_id',flat=True))
            print(len(subordinate_user_ids))
    else:
        subordinate_user_ids = list(AgentSupervisorSeasonMap.objects.filter(season_id=season_id).values_list('agent_id',flat=True))
print(subordinate_user_ids)
subordinate_user_ids = list(set(subordinate_user_ids))
# subordinate_user_ids = list(UserProfile.objects.filter(user_id__in=subordinate_user_ids, user_type_id=user_type_id).values_list("user_id", flat=True))

if AgentFarmerMap.objects.filter(agent_id__in=subordinate_user_ids, farmer__season_id=season_id).exists():
    print('exetessoko')
    filter_farmer_ids = list(AgentFarmerMap.objects.filter(agent_id__in=subordinate_user_ids, farmer__season_id=season_id).values_list("farmer__farmer_id", flat=True))
   
    farmer_ids = list(Harvest.objects.filter(sowing__season_id=season_id).values_list("sowing__farmer__id", flat=True))
#     print(farmer_ids)
    filtered_farmer_ids = list(FarmerClusterSeasonMap.objects.filter(season_id=season_id, farmer_id__in=filter_farmer_ids).exclude(farmer_id__in=farmer_ids).values_list("farmer__id", flat=True))
#     print(filtered_farmer_ids)
    # filter only the cluster value deducted

    if "selected_cluster_ids" in data_dict_from_interface:
        print('yesssss')
        cluster_farmer_obj = FarmerClusterSeasonMap.objects.filter(cluster_id__in=data_dict_from_interface['selected_cluster_ids'], season_id=season_id)
    else:
        cluster_farmer_obj = FarmerClusterSeasonMap.objects.filter(season_id=season_id)

    cluster_farmer_values = list(cluster_farmer_obj.filter(farmer_id__in=filter_farmer_ids, season_id=season_id).exclude(farmer_id__in=farmer_ids).values_list("id","cluster__name", "cluster__id", "farmer_id", "farmer__first_name", "farmer__last_name", "farmer__village__name", "seasonal_farmer_code",))
    cluster_farmer_columns = ["id","cluster_name","cluster_id","farmer_id","farmer_first_name","farmer_last_name","farmer_village_name","farmer_code",]
    cluster_farmer_df = pd.DataFrame(cluster_farmer_values, columns=cluster_farmer_columns)
    
    file='MarksData.xlsx'
    cluster_farmer_df.to_excel(file)
    
    print(cluster_farmer_df)
    print("nisciwbcdibwdubwduqbwdoiu")
    # filtering sowing objs based on option selected
    sowing_obj = Sowing.objects.filter(season_id=season_id)

#     # if 'date_greater' in data_dict_from_interface:
#     #     sowing_obj = sowing_obj.filter(sowing_date__gte=data_dict_from_interface['date_greater'])
#     # if 'date_lesser' in data_dict_from_interface:
#     #     sowing_obj = sowing_obj.filter(sowing_date__lte=data_dict_from_interface['date_lesser'])

    sowing_values = list(sowing_obj.filter(farmer_id__in=filtered_farmer_ids, season_id=season_id, cultivation_phase_id=2).values_list("id","farmer_id","sowing_date","area","cultivation_phase_id","cultivation_phase__name",))
    sowing_columns = ["sowing_id","farmer_id","sowing_date","area","cultivation_phase_id","cultivation_phase_name"]
    sowing_df = pd.DataFrame(sowing_values, columns=sowing_columns)
    
    file='MarksData1.xlsx'
    sowing_df.to_excel(file)
    
    

    # converted_today = datetime.datetime.strptime(, '%Y-%m-%d')
    today = datetime.datetime.now().date()
    sowing_df["crop_age"] = today - sowing_df["sowing_date"]
    sowing_df["crop_age"] = sowing_df["crop_age"].astype("timedelta64[D]")
    sowing_df["crop_age"] = sowing_df["crop_age"].astype(int)
    farmer_sowing_merged_df = pd.merge(cluster_farmer_df,sowing_df,left_on="farmer_id",right_on="farmer_id",how="left")



    farmer_cluster_map_ids = list(FarmerClusterSeasonMap.objects.filter(season_id=season_id).exclude(farmer_id__in=farmer_ids).values_list("id", flat=True))
    agent_farmer_map_values = list(AgentFarmerMap.objects.filter(farmer_id__in=farmer_cluster_map_ids).values_list("id", "farmer__farmer_id", "agent__first_name", "agent__last_name"))
    agent_farmer_map_columns = [ "agent_map_id", "farmer_id", "agent_first_name", "agent_last_name",]
    agent_farmer_map_df = pd.DataFrame(agent_farmer_map_values, columns=agent_farmer_map_columns)

    df = pd.merge(farmer_sowing_merged_df,agent_farmer_map_df,left_on="farmer_id",right_on="farmer_id",how="left",)
    df = df.drop(["id","cluster_id","farmer_id","sowing_id","cultivation_phase_id","agent_map_id",], axis=1,)
    df = df.rename(columns = {'cluster_name' : 'Cluster Name', 'farmer_first_name' : 'Farmer First Name', 'farmer_last_name' : 'Farmer Last Name',
   'farmer_village_name' : 'Farmer Village Name', 'farmer_code' : 'Farmer Code', 'sowing_date' : 'Sowing Date', 'area' : 'Area',
   'cultivation_phase_name' : 'Cultivation Phase Name', 'crop_age' : 'Crop Age', 'agent_first_name' : 'Agent First Name',
   'agent_last_name' : 'Agent Last Name'})
    df = df.fillna(0)
    print("===========================", df)
    writer = pd.ExcelWriter(str("static/media/") + "Non_supplied_farmers.xlsx", engine="xlsxwriter")

    # assigning that sheet to obj
    workbook = writer.book

    merge_format = workbook.add_format({
            "bold": 1,
            "border": 1,
            "align": "center",
            "valign": "vcenter",
            "fg_color": "yellow",
        }
    )

    header_format = workbook.add_format({"fg_color": "#D7E4BC"})
    format1 = workbook.add_format({"num_format": "#,##0.00"})
    if user_type_id == 5:
        subordinate_user_ids = [user_id]
        clusters = UserClusterMap.objects.filter(user_id=user_id).values_list("cluster__name", flat=True)
        df = df[df["Cluster Name"].isin(clusters)]
    for cluster in df["Cluster Name"].unique():
        cluster_based_df = df[df["Cluster Name"] == cluster]
        cluster_based_age_based_df = cluster_based_df[cluster_based_df["Crop Age"] > 35]
        cluster_based_age_based_df.to_excel(writer, sheet_name=str(cluster), startrow=1)
        cluster = writer.sheets[str(cluster)]
        cluster.merge_range("A1:L1", "farmers", merge_format)
        format1 = workbook.add_format({"num_format": "#,##0.00"})
        cluster.set_column("B:B", 18, format1)
        cluster.set_column(0, 11, 20)

        # Write the column headers with the defined format.
        for col_num, value in enumerate(df.columns.values):
            cluster.write(0, col_num + 1, value, header_format)
    writer.save()
    print('ok...')
    try:
        image_path = str("static/media/") + "Non_supplied_farmers.xlsx"
        with open(image_path, "rb") as image_file:
            encoded_image = b64encode(image_file.read())
            document["excel"] = encoded_image
    except Exception as err:
        print(err)
    document["data"] = df.to_dict("r")


data_dict_from_interface
idzzz:
6
220
[71, 73, 72, 166, 61, 59, 168, 56, 44, 32, 41, 34, 37, 31, 48, 46, 87, 123, 154, 173, 85, 172, 145, 88, 40, 111, 51, 50, 55, 74, 165, 146, 138, 167, 164, 47, 60, 103, 35, 49, 144, 169, 122, 180, 110, 120, 153, 121, 91, 142, 141, 179, 178, 184, 43, 65, 135, 170, 143, 181, 201, 183, 171, 199, 174, 124, 58, 202, 194, 182, 196, 206, 195, 90, 210, 134, 200, 139, 175, 176, 63, 198, 197, 77, 203, 177, 211, 219, 220]
exetessoko
         id cluster_name  cluster_id  farmer_id farmer_first_name  \
0     12850    Shivapura          20       2656         Siddaraju   
1     12849    Shivapura          20       2657         H S suma    
2     11675     Mattkere          16       2669           Nagamma   
3     13276      Udigala          19       2696    Somashekarappa   
4     17130    Gopalpura           1       2709      Gopalashetty   
...     ...          ...         ...        ...               ...   
2698  17853        Belur          22      15917        

<ipython-input-16-e3bb6a7788de>:138: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  document["data"] = df.to_dict("r")


In [18]:
from base64 import b64encode, b64decode
document = {}
try:
    image_path = str("static/media/") + "harvest_wise_reports.xlsx"
    with open(image_path, "rb") as image_file:
        encoded_image = b64encode(image_file.read())
        document["excel"] = encoded_image
except Exception as err:
    print(err)
# data = fine_df.to_dict('r')




# harvest wise range report

In [4]:
import pandas as pd
import datetime
from base64 import b64encode, b64decode

season_id=4

t1 = datetime.datetime.now()
print('-----script started at : {} -----'.format(t1))
harvest_values = Harvest.objects.filter(sowing__cultivation_phase_id=2, sowing__season_id=season_id).order_by('sowing_id','date_of_harvest').values_list(
    'sowing_id', 'sowing__farmer_id', 'date_of_harvest', 'value', 'unit__name', 'sowing__sowing_date','sowing__farmer__aadhaar_number', 'sowing__farmer__village__name','sowing__area_calculated_via_geo_fencing','sowing__water_source__name')
harvest_columns = ['sowing_id', 'farmer_id', 'date_of_harvest', 'value', 'unit', 'sowing_date', 'farmer_aadhaar_number','farmer_village','area_calculated_via_geo_fencing','water_source']
harvest_df = pd.DataFrame(list(harvest_values), columns=harvest_columns)

harvest_df['harvest_days'] = (harvest_df['date_of_harvest'] - harvest_df['sowing_date']).dt.days
print('harvest df created')
harvest_days_group = {
    1: {'min': 0, 'max': 61},
    2: {'min': 62, 'max': 68},
    3: {'min': 69, 'max': 75},
    4: {'min': 76, 'max': 82},
    5: {'min': 83, 'max': 89},
    6: {'min': 90, 'max': 96},
    7: {'min': 97, 'max': 103},
    8: {'min': 104, 'max': 110},
    9: {'min': 111, 'max': 117},
    10: {'min': 118, 'max': 200},
}

harvest_count_dict = {
    1: ('First Harvest Date', '1st Kg'),
    2: ('2nd Harvest Date', '2nd Kg'),
    3: ('3rd Harvest Date', '3rd Kg'),
    4: ('4th Harvest Date', '4th Kg'),
    5: ('5th Harvest Date', '5th Kg'),
    6: ('6th Harvest Date', '6th Kg'),
    7: ('7th Harvest Date', '7th Kg'),
    8: ('8th Harvest Date', '8th Kg'),
    9: ('9th Harvest Date', '9th Kg'),
    10: ('10th Harvest Date', '10th Kg'),
}

harvest_df = harvest_df.assign(**{'harvest_count': None})
for i in harvest_days_group:
    harvest_df.loc[(harvest_df['harvest_days'] >= harvest_days_group[i]['min']) & (harvest_df['harvest_days'] <= harvest_days_group[i]['max']), 'harvest_count'] = i
for i in harvest_count_dict:
    harvest_df[harvest_count_dict[i][1]] = 0
    harvest_df[harvest_count_dict[i][0]] = 0
print('created new fields')
for sowing_id in harvest_df['sowing_id'].unique():
    sowing_based_harvest_df = harvest_df[harvest_df['sowing_id']==sowing_id]
    har_count_based_sowing = sowing_based_harvest_df.groupby("harvest_count")["value"].sum().to_dict()
    for harvest_count in har_count_based_sowing:
        harvest_df.loc[harvest_df['sowing_id'] == sowing_id, harvest_count_dict[harvest_count][1]] = har_count_based_sowing[harvest_count]
        min_date = harvest_df[(harvest_df['sowing_id'] == sowing_id) & (harvest_df['harvest_count'] == harvest_count)].date_of_harvest.min()
        harvest_df.loc[harvest_df['sowing_id'] == sowing_id, harvest_count_dict[harvest_count][0]] = min_date

print('all the sowing values calculated and filled in fields')
harvest_df = harvest_df.drop_duplicates('sowing_id')
farmer_values = Farmer.objects.filter(id__in=harvest_df['farmer_id']).values_list('id', 'first_name', 'last_name','code')
farmer_columns = ['id', 'farmer_first_name', 'farmer_last_name', 'farmer_code']
farmer_df = pd.DataFrame(list(farmer_values), columns=farmer_columns)
#agent_values = AgentFarmerMap.objects.filter(farmer__farmer_id__in=harvest_df['farmer_id'], farmer__season_id=season_id).values_list('farmer_id', 'agent_id', 'agent__first_name', 'agent__last_name')
###########################
agent_values = AgentFarmerMap.objects.filter(farmer__farmer_id__in=harvest_df['farmer_id'], farmer__season_id=season_id).values_list('farmer__farmer_id', 'agent_id', 'agent__first_name', 'agent__last_name')
agent_columns = ['farmer_id', 'agent_id', 'agent_first_name', 'agent_last_name']

print('created farmer df for farmer details')

agent_df = pd.DataFrame(list(agent_values), columns=agent_columns)
#harvest_df = harvest_df.merge(farmer_df, how='inner', left_on='farmer_id', right_on='id').merge(agent_df, how='inner',left_on='farmer_id',right_on='farmer_id')

harvest_df = harvest_df.merge(farmer_df, how='left', left_on='farmer_id', right_on='id').merge(agent_df, how='left',left_on='farmer_id',right_on='farmer_id')
harvest_df = harvest_df.fillna(0)

print('merged farmer df with harvest df')

harvest_table_list = list(Harvest.objects.filter(sowing__cultivation_phase_id=2, sowing__season_id=season_id).values_list( "sowing__farmer__code", "sowing__farmer__id", "sowing__farmer__first_name", "sowing__farmer__last_name", "sowing__season__name", "value", "date_of_harvest", "sowing__area", "sowing__farmer__cluster__name",))
column_list = [ "farmer_code", "farmer_id", "farmer_first_name", "farmer_last_name", "season", "yeald", "date_of_harvest", "tp_area", "cluster_name",]
harvest_data_frame = pd.DataFrame(harvest_table_list, columns=column_list)

# agent_data
# agent_farmer_list = list(AgentFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame["farmer_code"], farmer__season_id=season_id).values_list("farmer__farmer__code", "agent__first_name"))
##########################
agent_farmer_list = list(AgentFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame["farmer_code"], farmer__season_id=season_id).values_list("farmer__seasonal_farmer_code", "agent__first_name"))
agent_farmer_columns = ["farmer_code", "agent_name"]
agent_farmer_df = pd.DataFrame(agent_farmer_list, columns=agent_farmer_columns)

# supervisor_data
supervisor_farmer_list = list(UserFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame["farmer_code"], farmer__season_id=season_id).values_list("farmer__farmer__code", "officer__username"))
supervisor_farmer_colums = ["farmer_code", "supervisor_name"]
supervisor_farmer_df = pd.DataFrame(supervisor_farmer_list, columns=supervisor_farmer_colums)
0
# merged_data
farmer_agent_map_df = pd.merge(harvest_data_frame,agent_farmer_df,left_on="farmer_code",right_on="farmer_code")
supervisor_fatmer_agent_map_df = pd.merge(farmer_agent_map_df, supervisor_farmer_df, left_on="farmer_code", right_on="farmer_code")

# sum yeald and find last harvest date finally merge with supervisor_fatmer_agent_map_df
grouped_df = (supervisor_fatmer_agent_map_df.groupby(["farmer_code"]).agg({"yeald": "sum", "date_of_harvest": "max"}).rename(columns={"yeald": "total_yeild", "date_of_harvest": "last_harvest_date"}))
final_supervisor_fatmer_agent_map_df = pd.merge(supervisor_fatmer_agent_map_df,grouped_df,left_on="farmer_code",right_index=True,)
final_supervisor_fatmer_agent_map_df = final_supervisor_fatmer_agent_map_df.drop(columns=["yeald", "date_of_harvest"])
final_supervisor_fatmer_agent_map_df = (final_supervisor_fatmer_agent_map_df.drop_duplicates())

print('final consolidated df created')
# final_df
cons_df = final_supervisor_fatmer_agent_map_df.reindex(columns=["farmer_id","cluster_name","last_harvest_date","tp_area","total_yeild","supervisor_name"])

harvest_df = harvest_df.fillna(0)
harvest_df['total'] = harvest_df['1st Kg'] + harvest_df['2nd Kg'] + harvest_df['3rd Kg']+ harvest_df['4th Kg']+ harvest_df['5th Kg']+ harvest_df['6th Kg']+ harvest_df['7th Kg']+ harvest_df['8th Kg']+ harvest_df['9th Kg']+ harvest_df['10th Kg']
df = pd.merge(harvest_df, cons_df, left_on="farmer_id", right_on="farmer_id", how="left")
df.drop_duplicates(subset="farmer_id", keep="first", inplace=True)
df = df.drop(['sowing_id', 'farmer_id', 'unit', 'value', 'date_of_harvest'], axis=1)
df = df.fillna(0)
df["avg_yield_per_acre"] = round(df['total_yeild']/df['tp_area'],2)
# writer = pd.ExcelWriter(str('/opt/bin/ccgb/ccgb_admin/static/media/') + 'harvest_wise_reports.xlsx', engine='xlsxwriter')
writer = pd.ExcelWriter(str("static/media/") + 'harvest_wise_reports.xlsx', engine='xlsxwriter')

# creating excel sheet with name
df1 = df[df['total'] == df['total_yeild']]
df1['S.No'] = range(1, 1 + len(df1))
df1 = df1[['S.No', 'supervisor_name','agent_first_name', 'agent_last_name','farmer_first_name', 'farmer_last_name', 'farmer_code', 'farmer_aadhaar_number', 'farmer_village',
        'First Harvest Date', '1st Kg','2nd Harvest Date', '2nd Kg', '3rd Harvest Date', '3rd Kg', '4th Harvest Date', '4th Kg',
        '5th Harvest Date', '5th Kg','6th Harvest Date', '6th Kg', '7th Harvest Date', '7th Kg', '8th Harvest Date', '8th Kg',
        '9th Harvest Date', '9th Kg', '10th Harvest Date', '10th Kg', 'total', 'tp_area','total_yeild', 'avg_yield_per_acre']]
df1.to_excel(writer, sheet_name='Sheet1', index=False, startrow=1)
print('correct harvest filter created')

df2 = df[df['total'] != df['total_yeild']]
df2['S.No'] = range(1, 1 + len(df2))
df2 = df2[['S.No', 'supervisor_name','agent_first_name', 'agent_last_name','farmer_first_name', 'farmer_last_name', 'farmer_code', 'farmer_aadhaar_number', 'farmer_village',
        'First Harvest Date', '1st Kg', '2nd Harvest Date', '2nd Kg', '3rd Harvest Date', '3rd Kg', '4th Harvest Date', '4th Kg', '5th Harvest Date', '5th Kg','6th Harvest Date',
        '6th Kg', '7th Harvest Date', '7th Kg', '8th Harvest Date', '8th Kg','9th Harvest Date', '9th Kg','10th Harvest Date', '10th Kg', 'total', 'tp_area', 'total_yeild','avg_yield_per_acre']]
df2.to_excel(writer, sheet_name='Sheet2', index=False, startrow=1)
# df.insert(1, 'S.no', range(1, 1 + len(df)))
print('incorrect harvest filter created')
# assigning that sheet to obj
workbook = writer.book
worksheet1 = writer.sheets['Sheet1']
worksheet2 = writer.sheets['Sheet2']
merge_format = workbook.add_format({
    'bold': 1,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': 'yellow'})

# Merge 3 cells.
# format = "%Y-%m-%d %H:%M:%S %Z%z"
# now_utc = datetime.datetime.now(timezone('UTC'))
# now_asia = now_utc.astimezone(timezone('Asia/Kolkata'))
# current_time = now_asia.strftime("%b %d %Y %H:%M:%S")

name = 'Harvest Range wise report' + ' (Updated on: ' + str(date) +')'
worksheet1.merge_range('A1:AG1', name, merge_format)
worksheet2.merge_range('A1:AG1', 'Late harvest data (some entries will not be included for these entries)', merge_format)
format1 = workbook.add_format({'num_format': '#,##0.00'})

# Set the column width and format.
worksheet1.set_column('B:B', 18, format1)
worksheet1.set_column(0, 32, 20)

# Set the column width and format.
worksheet2.set_column('B:B', 18, format1)
worksheet2.set_column(0, 32, 20)

# Add a header format.
header_format = workbook.add_format({'fg_color': '#D7E4BC'})

# Write the column headers with the defined format.
for col_num, value in enumerate(df1.columns.values):
    worksheet1.write(0, col_num + 1, value, header_format)

# Write the column headers with the defined format.
for col_num, value in enumerate(df2.columns.values):
    worksheet2.write(0, col_num + 1, value, header_format)

writer.save()
print('excel generated and stored')
t2 = datetime.datetime.now()
print('script completed with duration of : {}'.format(t2-t1))


-----script started at : 2022-09-16 12:47:22.378841 -----
harvest df created
created new fields
all the sowing values calculated and filled in fields
created farmer df for farmer details
merged farmer df with harvest df
final consolidated df created


<ipython-input-4-c7b8041b0e82>:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['S.No'] = range(1, 1 + len(df1))


correct harvest filter created


<ipython-input-4-c7b8041b0e82>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['S.No'] = range(1, 1 + len(df2))


incorrect harvest filter created


NameError: name 'date' is not defined

In [66]:
agent_values = AgentFarmerMap.objects.filter(farmer__farmer_id__in=harvest_df['farmer_id'], farmer__season_id=season_id).values_list('farmer_id', 'agent_id', 'agent__first_name', 'agent__last_name')
agent_columns = ['farmer_id', 'agent_id', 'agent_first_name', 'agent_last_name']

print('created farmer df for farmer details')

agent_df = pd.DataFrame(list(agent_values), columns=agent_columns)
agent_df.to_excel("ag.xlsx")

created farmer df for farmer details


In [9]:
agent_farmer_list = list(AgentFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame["farmer_code"], farmer__season_id=season_id).values_list("farmer__farmer__code", "agent__first_name"))
agent_farmer_columns = ["farmer_code", "agent_name"]
agent_farmer_df = pd.DataFrame(agent_farmer_list, columns=agent_farmer_columns)
agent_farmer_df

,farmer_code,agent_name
0,22MNA001,Basavanna M
1,21NPD052,Ranga Swamy
2,21NPD071,Ranga Swamy
3,21NPD012,Ranga Swamy
4,21TKE057,Muruga_Terakanambi
...,...,...
4062,22MMC103,Nagaraju Naika
4063,22HBB412,Siddananjappa _ Beluru
4064,22MME026,Venkataraju
4065,22MHI180,Puttamallappa


In [71]:
# agent_farmer_list = list(AgentFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame["farmer_code"], farmer__season_id=season_id).values_list("farmer__farmer__code", "agent__first_name"))
# col1=["farmer__code","agent__first_name"]
# df1=pd.DataFrame(agent_farmer_list,columns=col1)
# supervisor_farmer_list = list(UserFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame["farmer_code"], farmer__season_id=season_id).values_list("farmer__farmer__code", "officer__username"))
# col2=["farmer__code","officer__username"]
# df2=pd.DataFrame(supervisor_farmer_list,columns=col2)
# df2
# # harvest_df = harvest_df.merge(farmer_df, how='inner', left_on='farmer_id', right_on='id').merge(agent_df, how='inner',left_on='farmer_id',right_on='farmer_id')

# # harvest_df = harvest_df.merge(df1, how='inner', left_on='farmer_code', right_on='farmer__code')
# # .merge(df2, how='inner',left_on='farmer_code',right_on='farmer__code')
# harvest_df

In [20]:
from django.shortcuts import redirect, get_object_or_404

obj = get_object_or_404(User, id=219)
obj


<User: 99944433888>

In [41]:
import time
m = PositionPositionUserMap.objects.filter(user__userprofile__user_type_id=1).values_list()
print("object---------->",m)
a=list(PositionPositionUserMap.objects.filter(position__positionpositionusermap__user_id=23).values_list("id","position__name","user__username"))
print("--------------->hahaha",a)

b={"position":{}}
for i in m:
    print("--------------->",i.position.name)
   


object----------> <QuerySet [(10, 11, 78, True, datetime.datetime(2020, 1, 1, 10, 56, 24, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 10, 56, 24, tzinfo=<UTC>)), (19, 21, 116, True, datetime.datetime(2020, 1, 1, 10, 56, 24, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 10, 56, 24, tzinfo=<UTC>)), (25, 27, 159, True, datetime.datetime(2020, 1, 1, 10, 56, 24, tzinfo=<UTC>), datetime.datetime(2020, 1, 1, 10, 56, 24, tzinfo=<UTC>))]>
--------------->hahaha [(2, 'AFS02', 'naveen')]


AttributeError: 'tuple' object has no attribute 'position'

In [39]:
user = authenticate(username="Naveen", password="1234")
print(user.username
     )

Naveen


In [5]:
Question.objects.filter().order_by('ordinal')

<QuerySet [<Question: 15 - What is the previous crop cultivated in this filed>, <Question: 17 - Crop on the North>, <Question: 8 - Seed Issue Date>, <Question: 24 - Crop on the North>, <Question: 14 - Depth of water body>, <Question: 21 - Date of Transplantation>, <Question: 30 - Distance between two rows>, <Question: 1 - Gender>, <Question: 28 - What is the previous crop cultivated in this filed>, <Question: 16 - When was it Cultivated>, <Question: 2 - Education>, <Question: 25 - Crop on the East>, <Question: 31 - Distance between two plants >, <Question: 22 - Transplanting method>, <Question: 9 - Seed label>, <Question: 29 - When was it Cultivated>, <Question: 18 - Crop on the East>, <Question: 23 - Number of Seedlings Transplanted>, <Question: 3 - Caste>, <Question: 10 - Nursery Raising Approach>, '...(remaining elements truncated)...']>

In [3]:
agents_user_id = UserProfile.objects.filter(user_type_id=6,).values_list("user_id", flat=True)
print(agents_user_id)
user = User.objects.filter(id__in=agents_user_id).order_by("id")
print(user)
user_values = user.values_list("id", "first_name", "last_name", "first_name", "userhierarchymap__superior_id", 'userhierarchymap__season_id')
user_values

<QuerySet [33, 43, 154, 37, 39, 35, 60, 107, 71, 109, 42, 72, 36, 38, 67, 70, 69, 153, 52, 51, '...(remaining elements truncated)...']>
<QuerySet [<User: 8217357218>, <User: 9980310639>, <User: 1111111111>, <User: 9945675534>, <User: 8548092445>, <User: 9108390550>, <User: 9845488754>, <User: 9632076751>, <User: 9743908690>, <User: 9900906681>, <User: 9611664134>, <User: 9632820922>, <User: 7406990508>, <User: 9880838227>, <User: 7026921888>, <User: 9164262016>, <User: 7259919705>, <User: 9743165940>, <User: 9741650872>, <User: 9626341827>, '...(remaining elements truncated)...']>


<QuerySet [(31, 'Arasashetty', 'Kongalashetty', 'Arasashetty', None, None), (32, 'Bangaranaika', 'Siddanaika', 'Bangaranaika', None, None), (33, 'Dileep', 'Dileep', 'Dileep', None, None), (34, 'H N Papanna', 'Nanjappa', 'H N Papanna', None, None), (35, 'Madappa', 'Gurappa', 'Madappa', None, None), (36, 'Madappa G B', 'Bellappa', 'Madappa G B', None, None), (37, 'Mahadevaprasad B S', 'Siddappa', 'Mahadevaprasad B S', None, None), (38, 'H B Mahadevaswamy', 'H M Basavanna', 'H B Mahadevaswamy', None, None), (39, 'S Manikanta ', 'Shivalingappa', 'S Manikanta ', None, None), (40, 'R Ravi ', 'Rajaiah', 'R Ravi ', None, None), (41, 'Shivamahadevu', 'Mudalagiriah', 'Shivamahadevu', None, None), (42, 'Chandrashekhararadya', 'Channaveeraradya', 'Chandrashekhararadya', None, None), (43, 'Gopala H', 'Hanumaiah', 'Gopala H', None, None), (44, 'M Kumaraswamy', 'Mudddappa', 'M Kumaraswamy', None, None), (45, 'Mahesha', 'Basavarajappa', 'Mahesha', None, None), (46, 'Manjesh M', 'Mallappa', 'Manjesh M'

In [8]:
import pandas as pd
Procurement.objects.filter(procurement_group__procurement_date__range=["2020-5-30", "2020-7-30"])

<QuerySet [<Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, '...(remaining elements truncated)...']>

In [15]:
Procurement.objects.filter(procurement_group__procurement_date__range=["2020-5-30", "2020-7-30"]).order_by("procurement_group__procurement_date")


<QuerySet [<Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, <Procurement: Procurement object>, '...(remaining elements truncated)...']>

In [17]:
a=Procurement.objects.filter(procurement_group__procurement_date__range=["2020-5-30", "2020-6-2"]).order_by("procurement_group__procurement_date").values_list( "procurement_group__procurement_date", "ticket_number", "procurement_transaport_incharge_kyc__aadhar_number", 
                "procurement_group__agent__id", "procurement_group__agent__first_name", "vehicle_number", 
                "loaded_vehicle_weight", "empty_vehicle_weight", "produce_gross_weight", "gunnybag_count",
                "gunnybag_weight", "moisture", "other_deduction", "str_weight", "procurement_group__produce_net_weight",
                "procurement_group__price_per_unit", "procurement_group__agent_price_deduction", 
                "procurement_group__payment_to_wallet", "procurement_group__payment_to_agent")


procurement_columns = [ "Date", "Ticket No", "ID/Company", "agent_id", "Agent Name", 
                       "Vehicle No", "Gross Wt/kg", "Tare Wt/kg","Net Wt/kg", "Bag No", 
                       "Bag Deduct", "moisture Deduct", "other_deduction", "str_weight", 
                       "Final Weight", "Total prices/Per kg", 
                       "Deduction Price/kg", "Deduction/kg","payment_to_agent"]
procurement_df = pd.DataFrame(a, columns=procurement_columns)

a=procurement_df.to_dict("r")
procurement_df
a

<ipython-input-17-9245e9df6bd6>:16: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  a=procurement_df.to_dict("r")


[{'Date': datetime.date(2020, 5, 30),
  'Ticket No': '2020053001002',
  'ID/Company': '671102833133',
  'agent_id': 71,
  'Agent Name': 'K Basappa',
  'Vehicle No': 'KA108663',
  'Gross Wt/kg': Decimal('980.00'),
  'Tare Wt/kg': Decimal('600.00'),
  'Net Wt/kg': Decimal('380.00'),
  'Bag No': 8,
  'Bag Deduct': Decimal('2.00'),
  'moisture Deduct': Decimal('0.00'),
  'other_deduction': Decimal('0.00'),
  'str_weight': Decimal('496.00'),
  'Final Weight': Decimal('378.00'),
  'Total prices/Per kg': Decimal('8.70'),
  'Deduction Price/kg': Decimal('1.50'),
  'Deduction/kg': Decimal('567.00'),
  'payment_to_agent': Decimal('2721.60')},
 {'Date': datetime.date(2020, 5, 30),
  'Ticket No': '2020053001001',
  'ID/Company': '870728285806',
  'agent_id': 110,
  'Agent Name': 'Kemparaju_Nanjangud',
  'Vehicle No': 'KA09C9581',
  'Gross Wt/kg': Decimal('1280.00'),
  'Tare Wt/kg': Decimal('1160.00'),
  'Net Wt/kg': Decimal('120.00'),
  'Bag No': 3,
  'Bag Deduct': Decimal('1.00'),
  'moisture Ded

In [16]:
agent_cluster = UserClusterMap.objects.filter(season_id=2, user__userprofile__user_type_id=6).values_list('user_id', 'cluster__name')
agent_cluster_col = ['agent_id', 'cluster_name']
agent_cluster_df = pd.DataFrame(agent_cluster, columns=agent_cluster_col)
agent_cluster_df

,agent_id,cluster_name
0,33,Gopalpura
1,45,Nanjangud
2,52,Nanjangud
3,42,Yadalaya
4,38,Gopalpura
...,...,...
59,120,Terakanambi
60,137,Sargur
61,138,Terakanambi
62,122,Sargur


In [32]:
procurement_cluster_df = pd.merge(procurement_df, agent_cluster_df, left_on='agent_id', right_on='agent_id', how='left')
procurement_cluster_df

,Date,Ticket No,ID/Company,agent_id,Agent Name,Vehicle No,Gross Wt/kg,Tare Wt/kg,Net Wt/kg,Bag No,...,other_deduction,str_weight,Final Weight,Total prices/Per kg,Deduction Price/kg,Deduction/kg,payment_to_agent,cluster_name_x,Bag weight Deduction/ Per kg,cluster_name_y
0,2020-05-30,2020053001002,671102833133,71,K Basappa,KA108663,980.00,600.00,380.00,8,...,0.00,496.00,378.00,8.70,1.50,567.00,2721.60,Bargi,0.2,Bargi
1,2020-06-11,2020061101002,671102833133,71,K Basappa,KA10A4351,2400.00,1260.00,1140.00,28,...,0.00,1107.00,1134.00,8.70,1.50,1701.00,8164.80,Bargi,0.2,Bargi
2,2020-06-19,2020061901001,671102833133,71,K Basappa,KA10A4351,3880.00,1260.00,2620.00,40,...,0.00,2567.00,2612.00,8.70,1.50,3918.00,18806.40,Bargi,0.2,Bargi
3,2020-06-20,2020062001001,671102833133,71,K Basappa,KA10A4351,3480.00,1240.00,2240.00,47,...,0.00,2210.00,2231.00,8.70,1.50,3346.50,16063.20,Bargi,0.2,Bargi
4,2020-06-25,2020062501009,671102833133,71,K Basappa,KA10A2151,4320.00,1300.00,3020.00,71,...,0.00,2938.00,3006.00,8.70,1.50,4509.00,21643.20,Bargi,0.2,Bargi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,2020-07-17,2020071701050,615238392052,117,M Mallikarjunaswamy,KA10A0605,1260.00,940.00,320.00,8,...,0.00,312.00,318.00,8.70,1.50,477.00,2289.60,Kodasoge,0.2,Kodasoge
1706,2020-07-22,2020072201060,615238392052,117,M Mallikarjunaswamy,KA100636,1840.00,840.00,1000.00,27,...,0.00,965.00,995.00,8.70,2.00,1990.00,6666.50,Kodasoge,0.2,Kodasoge
1707,2020-07-24,2020072401045,615238392052,117,M Mallikarjunaswamy,KA10A0636,1700.00,840.00,860.00,19,...,0.00,829.00,856.00,8.70,2.50,2140.00,5307.20,Kodasoge,0.2,Kodasoge
1708,2020-07-28,2020072801059,615238392052,117,M Mallikarjunaswamy,KA100636,2180.00,840.00,1340.00,35,...,0.00,1272.00,1333.00,8.70,1.50,1999.50,9597.60,Kodasoge,0.2,Kodasoge


In [33]:
procurement_cluster_df = pd.merge(procurement_df, agent_cluster_df)
procurement_cluster_df

,Date,Ticket No,ID/Company,agent_id,Agent Name,Vehicle No,Gross Wt/kg,Tare Wt/kg,Net Wt/kg,Bag No,...,moisture Deduct,other_deduction,str_weight,Final Weight,Total prices/Per kg,Deduction Price/kg,Deduction/kg,payment_to_agent,cluster_name,Bag weight Deduction/ Per kg
0,2020-05-30,2020053001002,671102833133,71,K Basappa,KA108663,980.00,600.00,380.00,8,...,0.00,0.00,496.00,378.00,8.70,1.50,567.00,2721.60,Bargi,0.2
1,2020-06-11,2020061101002,671102833133,71,K Basappa,KA10A4351,2400.00,1260.00,1140.00,28,...,0.00,0.00,1107.00,1134.00,8.70,1.50,1701.00,8164.80,Bargi,0.2
2,2020-06-19,2020061901001,671102833133,71,K Basappa,KA10A4351,3880.00,1260.00,2620.00,40,...,0.00,0.00,2567.00,2612.00,8.70,1.50,3918.00,18806.40,Bargi,0.2
3,2020-06-20,2020062001001,671102833133,71,K Basappa,KA10A4351,3480.00,1240.00,2240.00,47,...,0.00,0.00,2210.00,2231.00,8.70,1.50,3346.50,16063.20,Bargi,0.2
4,2020-06-25,2020062501009,671102833133,71,K Basappa,KA10A2151,4320.00,1300.00,3020.00,71,...,0.00,0.00,2938.00,3006.00,8.70,1.50,4509.00,21643.20,Bargi,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,2020-07-17,2020071701050,615238392052,117,M Mallikarjunaswamy,KA10A0605,1260.00,940.00,320.00,8,...,0.00,0.00,312.00,318.00,8.70,1.50,477.00,2289.60,Kodasoge,0.2
1706,2020-07-22,2020072201060,615238392052,117,M Mallikarjunaswamy,KA100636,1840.00,840.00,1000.00,27,...,0.00,0.00,965.00,995.00,8.70,2.00,1990.00,6666.50,Kodasoge,0.2
1707,2020-07-24,2020072401045,615238392052,117,M Mallikarjunaswamy,KA10A0636,1700.00,840.00,860.00,19,...,0.00,0.00,829.00,856.00,8.70,2.50,2140.00,5307.20,Kodasoge,0.2
1708,2020-07-28,2020072801059,615238392052,117,M Mallikarjunaswamy,KA100636,2180.00,840.00,1340.00,35,...,0.00,0.00,1272.00,1333.00,8.70,1.50,1999.50,9597.60,Kodasoge,0.2


In [28]:
procurement_df = procurement_cluster_df
procurement_df["Bag weight Deduction/ Per kg"] = procurement_df.apply(lambda x: "0.2",axis=1)
procurement_df


,Date,Ticket No,ID/Company,agent_id,Agent Name,Vehicle No,Gross Wt/kg,Tare Wt/kg,Net Wt/kg,Bag No,...,moisture Deduct,other_deduction,str_weight,Final Weight,Total prices/Per kg,Deduction Price/kg,Deduction/kg,payment_to_agent,cluster_name,Bag weight Deduction/ Per kg
0,2020-05-30,2020053001002,671102833133,71,K Basappa,KA108663,980.00,600.00,380.00,8,...,0.00,0.00,496.00,378.00,8.70,1.50,567.00,2721.60,Bargi,0.2
1,2020-06-11,2020061101002,671102833133,71,K Basappa,KA10A4351,2400.00,1260.00,1140.00,28,...,0.00,0.00,1107.00,1134.00,8.70,1.50,1701.00,8164.80,Bargi,0.2
2,2020-06-19,2020061901001,671102833133,71,K Basappa,KA10A4351,3880.00,1260.00,2620.00,40,...,0.00,0.00,2567.00,2612.00,8.70,1.50,3918.00,18806.40,Bargi,0.2
3,2020-06-20,2020062001001,671102833133,71,K Basappa,KA10A4351,3480.00,1240.00,2240.00,47,...,0.00,0.00,2210.00,2231.00,8.70,1.50,3346.50,16063.20,Bargi,0.2
4,2020-06-25,2020062501009,671102833133,71,K Basappa,KA10A2151,4320.00,1300.00,3020.00,71,...,0.00,0.00,2938.00,3006.00,8.70,1.50,4509.00,21643.20,Bargi,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,2020-07-17,2020071701050,615238392052,117,M Mallikarjunaswamy,KA10A0605,1260.00,940.00,320.00,8,...,0.00,0.00,312.00,318.00,8.70,1.50,477.00,2289.60,Kodasoge,0.2
1706,2020-07-22,2020072201060,615238392052,117,M Mallikarjunaswamy,KA100636,1840.00,840.00,1000.00,27,...,0.00,0.00,965.00,995.00,8.70,2.00,1990.00,6666.50,Kodasoge,0.2
1707,2020-07-24,2020072401045,615238392052,117,M Mallikarjunaswamy,KA10A0636,1700.00,840.00,860.00,19,...,0.00,0.00,829.00,856.00,8.70,2.50,2140.00,5307.20,Kodasoge,0.2
1708,2020-07-28,2020072801059,615238392052,117,M Mallikarjunaswamy,KA100636,2180.00,840.00,1340.00,35,...,0.00,0.00,1272.00,1333.00,8.70,1.50,1999.50,9597.60,Kodasoge,0.2


In [17]:
procurement_df["Quality Weight"] = procurement_df.apply(lambda x: "0", axis=1)
procurement_df


,Date,Ticket No,ID/Company,agent_id,Agent Name,Vehicle No,Gross Wt/kg,Tare Wt/kg,Net Wt/kg,Bag No,Bag Deduct,moisture Deduct,other_deduction,str_weight,Final Weight,Total prices/Per kg,Deduction Price/kg,Deduction/kg,payment_to_agent,Quality Weight
0,2020-05-30,2020053001002,671102833133,71,K Basappa,KA108663,980.00,600.00,380.00,8,2.00,0.00,0.00,496.00,378.00,8.70,1.50,567.00,2721.60,0
1,2020-05-30,2020053001001,870728285806,110,Kemparaju_Nanjangud,KA09C9581,1280.00,1160.00,120.00,3,1.00,0.00,0.00,1059.00,119.00,8.90,1.50,178.50,880.60,0
2,2020-06-01,2020060101003,679261428572,60,Shivakumar,NEW TRACTOR,6080.00,3800.00,2280.00,55,11.00,0.00,0.00,2373.00,2269.00,8.70,0.00,0.00,19740.30,0
3,2020-06-01,2020060101001,532638592963,46,Manjesh M,KA09C7987,2940.00,1280.00,1660.00,35,7.00,0.00,0.00,1722.00,1653.00,9.00,1.50,2479.50,12397.50,0
4,2020-06-01,2020060101002,363060978106,72,Maheshappa,KA10A1482,3100.00,1900.00,1200.00,24,5.00,0.00,0.00,1176.00,1195.00,8.70,1.50,1792.50,8604.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,2020-07-30,2020073001003,363060978106,72,Maheshappa,KA10A1482,4840.00,1940.00,2900.00,56,11.20,0.00,0.00,2806.00,2889.00,8.70,1.50,4333.50,20800.80,0
2079,2020-07-30,2020073001002,594721779775,73,Nagappa,KA10-9828,5560.00,2020.00,3540.00,65,13.00,0.00,0.00,3521.00,3527.00,8.70,1.50,5290.50,25394.40,0
2080,2020-07-30,2020073001001,601523324025,37,Mahadevaprasad B S,KA101404,9540.00,3720.00,5820.00,135,27.00,0.00,0.00,5742.00,5793.00,8.70,2.00,11586.00,38813.10,0
2081,2020-07-30,2020072901085,242094299338,91,Puttamallappa,KA41B0112,10160.00,3580.00,6580.00,120,24.00,0.00,0.00,6577.00,6556.00,9.00,0.00,0.00,59004.00,0


In [18]:
procurement_df["avg bag weight"] = procurement_df.apply(lambda x: round(x["Net Wt/kg"] / x["Bag No"],2 ), axis=1)
procurement_df["Payment Price/kg"] = procurement_df.apply(lambda x: (x["Total prices/Per kg"] - x["Deduction Price/kg"]), axis=1)
procurement_df['loss'] = ((procurement_df['str_weight'] - procurement_df['Final Weight']) / procurement_df['str_weight']) * 100
procurement_df['loss'] = procurement_df['loss'].astype(float).round(2)

procurement_df


,Date,Ticket No,ID/Company,agent_id,Agent Name,Vehicle No,Gross Wt/kg,Tare Wt/kg,Net Wt/kg,Bag No,...,str_weight,Final Weight,Total prices/Per kg,Deduction Price/kg,Deduction/kg,payment_to_agent,Quality Weight,avg bag weight,Payment Price/kg,loss
0,2020-05-30,2020053001002,671102833133,71,K Basappa,KA108663,980.00,600.00,380.00,8,...,496.00,378.00,8.70,1.50,567.00,2721.60,0,47.50,7.20,23.79
1,2020-05-30,2020053001001,870728285806,110,Kemparaju_Nanjangud,KA09C9581,1280.00,1160.00,120.00,3,...,1059.00,119.00,8.90,1.50,178.50,880.60,0,40.00,7.40,88.76
2,2020-06-01,2020060101003,679261428572,60,Shivakumar,NEW TRACTOR,6080.00,3800.00,2280.00,55,...,2373.00,2269.00,8.70,0.00,0.00,19740.30,0,41.45,8.70,4.38
3,2020-06-01,2020060101001,532638592963,46,Manjesh M,KA09C7987,2940.00,1280.00,1660.00,35,...,1722.00,1653.00,9.00,1.50,2479.50,12397.50,0,47.43,7.50,4.01
4,2020-06-01,2020060101002,363060978106,72,Maheshappa,KA10A1482,3100.00,1900.00,1200.00,24,...,1176.00,1195.00,8.70,1.50,1792.50,8604.00,0,50.00,7.20,-1.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,2020-07-30,2020073001003,363060978106,72,Maheshappa,KA10A1482,4840.00,1940.00,2900.00,56,...,2806.00,2889.00,8.70,1.50,4333.50,20800.80,0,51.79,7.20,-2.96
2079,2020-07-30,2020073001002,594721779775,73,Nagappa,KA10-9828,5560.00,2020.00,3540.00,65,...,3521.00,3527.00,8.70,1.50,5290.50,25394.40,0,54.46,7.20,-0.17
2080,2020-07-30,2020073001001,601523324025,37,Mahadevaprasad B S,KA101404,9540.00,3720.00,5820.00,135,...,5742.00,5793.00,8.70,2.00,11586.00,38813.10,0,43.11,6.70,-0.89
2081,2020-07-30,2020072901085,242094299338,91,Puttamallappa,KA41B0112,10160.00,3580.00,6580.00,120,...,6577.00,6556.00,9.00,0.00,0.00,59004.00,0,54.83,9.00,0.32


In [66]:
        bank_details = UserBankDetails.objects.filter(is_primary=True).values_list("user_id","bank","branch","ifsc_code","account_holder_name","account_number",)
        bank_columns = [ "user_id", "Bank name", "Branch", "IFSC Code", "BankAccount", "BankNumber",]
        bank_df = pd.DataFrame(bank_details, columns=bank_columns)
        bank_df

,user_id,Bank name,Branch,IFSC Code,BankAccount,BankNumber
0,10,ICICI Bank,Nanjakudu,ICIC0001722,Deepak,172201504657
1,11,ICICI Bank,NANJANGUD,ICIC0001722,Bhramarambika BS,172201504600
2,13,XXXX Bank,xxxxxx,xxxxxxx,xxxxx xxxx,111111111
3,18,State bank of india,sargur,SBIN40074,Siddappa,64066886880
4,21,xxxxxxx,xxxxxx,xxxxxxx,xxxxxx,11111111
...,...,...,...,...,...,...
120,122,Canara,Gundlupet,CNRB0004792,M Parameswaran,4792214000006
121,195,Federal bank,Handpost,FDRL0002172,Ravi M S,21720100007234
122,182,Canara Bank,"Maland college of engg, Hassan",CNRB0011405,Siddananjappa,14052200072591
123,181,Karnataka Grameena bank,Hangala,PKGB0012020,Mahadevappa (Jakkhalli),12020100001526


In [1]:
df = pd.merge(procurement_df, bank_df, left_on="agent_id", right_on="user_id", how="left")
df = df.drop(["agent_id", "user_id"],axis=1)
df

NameError: name 'pd' is not defined

In [12]:
df = df.fillna("-")
df

NameError: name 'df' is not defined

In [89]:
def convert_decimal(value):
    return round(value)

df["Gross Wt/kg"] = df.apply(lambda x: convert_decimal(x["Gross Wt/kg"]), axis=1)
df["Tare Wt/kg"] = df.apply(lambda x: convert_decimal(x["Tare Wt/kg"]), axis=1)
df["Net Wt/kg"] = df.apply(lambda x: convert_decimal(x["Net Wt/kg"]), axis=1)
df['Bag Deduct']=df.apply(lambda x: convert_decimal(x['Bag Deduct']), axis=1)
df["str_weight"] = df.apply(lambda x: convert_decimal(x["str_weight"]), axis=1)
df["Final Weight"] = df.apply(lambda x: convert_decimal(x["Final Weight"]), axis=1)
df

,Date,Ticket No,ID/Company,Agent Name,Vehicle No,Gross Wt/kg,Tare Wt/kg,Net Wt/kg,Bag No,Bag Deduct,...,avg bag weight,Payment Price/kg,loss,Bank name,Branch,IFSC Code,BankAccount,BankNumber,Total Amount/kg,Payment Amount/per kg
0,2020-05-30,2020053001002,671102833133,K Basappa,KA108663,980,600,380,8,2,...,47.50,7.20,23.79,State Bank of India,Gundlupet,SBIN0010364,Basappa K,30442134675,3288.6,2721.6
1,2020-05-30,2020053001001,870728285806,Kemparaju_Nanjangud,KA09C9581,1280,1160,120,3,1,...,40.00,7.40,88.76,Equitas,Gundlupet,ESFB0003011,Kemparaju,200001363606,1059.1,880.6
2,2020-06-01,2020060101003,679261428572,Shivakumar,NEW TRACTOR,6080,3800,2280,55,11,...,41.45,8.70,4.38,State Bank of India,Terakanambi,SBIN0040079,Shivakumar,40758161071,19740.3,19740.3
3,2020-06-01,2020060101001,532638592963,Manjesh M,KA09C7987,2940,1280,1660,35,7,...,47.43,7.50,4.01,HDFC Bank,Nanjangud,HDFC0002132,Manjesha M,50200052543112,14877.0,12397.5
4,2020-06-01,2020060101002,363060978106,Maheshappa,KA10A1482,3100,1900,1200,24,5,...,50.00,7.20,-1.62,State Bank of India,Gundlupet,SBIN0010364,Maheshappa,30447721034,10396.5,8604.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,2020-07-30,2020073001003,363060978106,Maheshappa,KA10A1482,4840,1940,2900,56,11,...,51.79,7.20,-2.96,State Bank of India,Gundlupet,SBIN0010364,Maheshappa,30447721034,25134.3,20800.8
2079,2020-07-30,2020073001002,594721779775,Nagappa,KA10-9828,5560,2020,3540,65,13,...,54.46,7.20,-0.17,Canara Bank,Bargi,CNRB0011713,Nagappa,17132210004273,30684.9,25394.4
2080,2020-07-30,2020073001001,601523324025,Mahadevaprasad B S,KA101404,9540,3720,5820,135,27,...,43.11,6.70,-0.89,Ujjivan Small Finance Bank,Gundlupet,UJVN0001629,Mahadevaprasad B S,1629120020000024,50399.1,38813.1
2081,2020-07-30,2020072901085,242094299338,Puttamallappa,KA41B0112,10160,3580,6580,120,24,...,54.83,9.00,0.32,Canara Bank,Sargur,CNRB0004343,Puttamallappa,4343101002316,59004.0,59004.0


In [7]:
def get_total_value(total_amount, deducted):
    value = total_amount - deducted
    return value 

df["Total Amount/kg"] = df.apply(lambda x: (x["Final Weight"] * x["Total prices/Per kg"]), axis=1)
df[["Total Amount/kg", "Deduction/kg"]]=df[["Total Amount/kg", "Deduction/kg"]].astype(float)
df["Payment Amount/per kg"] = df.apply(lambda x: get_total_value(x["Total Amount/kg"],x["Deduction/kg"]), axis=1)
df["Payment Amount/per kg"] = df['Payment Amount/per kg'].astype(float)
df


NameError: name 'df' is not defined

In [10]:
import math
def round_decimal(value):
    val = value
    x = 0

    if (float(val) % 1) >= 0.5:
        value = math.ceil(val)
    else:
        value = round(val)
    return value

df["Deduction/kg"]=df["Deduction/kg"].astype(float)
df["Payment Amount/per kg"] = df.apply(lambda x: round_decimal(x["Total Amount/kg"]-x["Deduction/kg"]), axis=1)
df = df[["Date","Ticket No","ID/Company","Agent Name","cluster_name","Vehicle No", "Gross Wt/kg", "Tare Wt/kg", "Net Wt/kg", "Bag No", "Bag weight Deduction/ Per kg", "Bag Deduct", "Quality Weight", "moisture Deduct", "other_deduction", "str_weight", "Final Weight", "Total prices/Per kg", "Total Amount/kg", "Deduction Price/kg", "Deduction/kg", "Payment Price/kg", "Payment Amount/per kg", "BankAccount", "Bank name", "Branch", "BankNumber", "IFSC Code", "avg bag weight","loss"]]

df

NameError: name 'df' is not defined

In [95]:
df['avg bag weight'] = df['avg bag weight'].astype(float)
df['Payment Price/kg'] = df['Payment Price/kg'].astype(float)
df['Deduction Price/kg'] = df['Deduction Price/kg'].astype(float)
df['Total prices/Per kg'] = df['Total prices/Per kg'].astype(float)
df['other_deduction'] = df['other_deduction'].astype(float)
df['Bag weight Deduction/ Per kg'] = df['Bag weight Deduction/ Per kg'].astype(float)
df['Quality Weight'] = df['Quality Weight'].astype(float)
df['moisture Deduct'] = df['moisture Deduct'].astype(float)

df


,Date,Ticket No,ID/Company,Agent Name,cluster_name,Vehicle No,Gross Wt/kg,Tare Wt/kg,Net Wt/kg,Bag No,...,Deduction/kg,Payment Price/kg,Payment Amount/per kg,BankAccount,Bank name,Branch,BankNumber,IFSC Code,avg bag weight,loss
0,2020-05-30,2020053001002,671102833133,K Basappa,Bargi,KA108663,980,600,380,8,...,567.0,7.2,2722,Basappa K,State Bank of India,Gundlupet,30442134675,SBIN0010364,47.50,23.79
1,2020-05-30,2020053001001,870728285806,Kemparaju_Nanjangud,Nanjangud,KA09C9581,1280,1160,120,3,...,178.5,7.4,881,Kemparaju,Equitas,Gundlupet,200001363606,ESFB0003011,40.00,88.76
2,2020-06-01,2020060101003,679261428572,Shivakumar,NaN,NEW TRACTOR,6080,3800,2280,55,...,0.0,8.7,19740,Shivakumar,State Bank of India,Terakanambi,40758161071,SBIN0040079,41.45,4.38
3,2020-06-01,2020060101001,532638592963,Manjesh M,Nanjangud,KA09C7987,2940,1280,1660,35,...,2479.5,7.5,12398,Manjesha M,HDFC Bank,Nanjangud,50200052543112,HDFC0002132,47.43,4.01
4,2020-06-01,2020060101002,363060978106,Maheshappa,Bargi,KA10A1482,3100,1900,1200,24,...,1792.5,7.2,8604,Maheshappa,State Bank of India,Gundlupet,30447721034,SBIN0010364,50.00,-1.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,2020-07-30,2020073001003,363060978106,Maheshappa,Bargi,KA10A1482,4840,1940,2900,56,...,4333.5,7.2,20801,Maheshappa,State Bank of India,Gundlupet,30447721034,SBIN0010364,51.79,-2.96
2079,2020-07-30,2020073001002,594721779775,Nagappa,Bargi,KA10-9828,5560,2020,3540,65,...,5290.5,7.2,25394,Nagappa,Canara Bank,Bargi,17132210004273,CNRB0011713,54.46,-0.17
2080,2020-07-30,2020073001001,601523324025,Mahadevaprasad B S,Gopalpura,KA101404,9540,3720,5820,135,...,11586.0,6.7,38813,Mahadevaprasad B S,Ujjivan Small Finance Bank,Gundlupet,1629120020000024,UJVN0001629,43.11,-0.89
2081,2020-07-30,2020072901085,242094299338,Puttamallappa,HD Kote,KA41B0112,10160,3580,6580,120,...,0.0,9.0,59004,Puttamallappa,Canara Bank,Sargur,4343101002316,CNRB0004343,54.83,0.32


In [9]:
df = df.rename(columns={"cluster_name" : 'Cluster Name' , "moisture Deduct" : 'Moisture Deduct' , "other_deduction" : 'Other Deduction' , "str_weight" : 'Str Weight' , "avg bag weight" : 'Avg Bag Weight' , "loss" : 'Loss' ,})
df

NameError: name 'df' is not defined

In [8]:
document={}
document["data"] = df.to_dict("r")
document

NameError: name 'df' is not defined

# doubt

In [7]:
df.index += 1
total = df.sum(numeric_only=True)
total.name = 'Total'
df = df.append(total.transpose())

NameError: name 'df' is not defined

In [6]:
writer = pd.ExcelWriter("static/media" + "/procurement_report.xlsx", engine="xlsxwriter")
writer

df.to_excel(writer, sheet_name="Sheet1", startrow=1)

NameError: name 'df' is not defined

In [5]:
workbook = writer.book
worksheet = writer.sheets["Sheet1"]
merge_format = workbook.add_format(
    {
        "bold": 1,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
        "fg_color": "yellow",
    }
)
merge_format
worksheet

NameError: name 'writer' is not defined

In [4]:
import datetime
date = datetime.datetime.now().date()
        # Merge 3 cells.
worksheet.merge_range(
    "A1:AE1",
    "Total Flower Puchase 2020 ( " + "2020-5-30" + " - " + "2020-7-30" + " )",
    merge_format,
)

format1 = workbook.add_format({"num_format": "#,##0.00"})


NameError: name 'worksheet' is not defined

In [3]:
# Set the column width and format.
worksheet.set_column("B:AE", 18, format1)
worksheet.set_column(0, 7, 20)

header_format = workbook.add_format({"fg_color": "#D7E4BC"})

NameError: name 'worksheet' is not defined

In [121]:
for col_num, value in enumerate(df.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)
writer.save()
try:
    image_path = str("static/media/") + "procurement_report.xlsx"
    with open(image_path, "rb") as image_file:
        encoded_image = b64encode(image_file.read())
        document["excel"] = encoded_image
except Exception as err:
    print(err)

In [129]:
def get_total_value(total_amount, deducted):
    value = total_amount - deducted
    return value
get_total_value(2000,65)

1935

In [34]:
def generate_issue_notice_bill_no():
    bill_obj = AgentIssueNoticeNumberCodeBank.objects.get(id=1)
    updated_bill_no = bill_obj.last_bill_number + 1
    bill_obj.last_bill_number = updated_bill_no
    bill_obj.save()
    return updated_bill_no
print(generate_issue_notice_bill_no())

70


In [21]:
import pickle
import pickletools

class Node:
    def __init__(self, data):
        print("self",data)
        self.data = data
        self.children = []
        print(self.data)

    def add_child(self, obj):
        print(obj)
        self.children.append(obj)
        print(self.children)


node = Node({"int": 1, "float": 2.0})
node1 = Node({"int": 5, "float": 6.0})
data = pickle.dumps(node)
data1 = pickle.dumps(node1)
data

self {'int': 1, 'float': 2.0}
{'int': 1, 'float': 2.0}
self {'int': 5, 'float': 6.0}
{'int': 5, 'float': 6.0}


b'\x80\x04\x95N\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x04Node\x94\x93\x94)\x81\x94}\x94(\x8c\x04data\x94}\x94(\x8c\x03int\x94K\x01\x8c\x05float\x94G@\x00\x00\x00\x00\x00\x00\x00u\x8c\x08children\x94]\x94ub.'

In [4]:
def get_active_season_id():
    season_obj = Season.objects.get(is_active=True)
    return season_obj.id
print(get_active_season_id())

4


In [75]:
values = HarvestLevel.objects.filter().order_by('ordinal').values_list('id', 'harvest_name')
columns = ['id', 'name']
df = pd.DataFrame(list(values), columns=columns)
data = df.to_dict('records')
data

[{'id': 1, 'name': '1'},
 {'id': 2, 'name': '2'},
 {'id': 3, 'name': '3'},
 {'id': 4, 'name': '4'},
 {'id': 5, 'name': '5'},
 {'id': 6, 'name': '6'},
 {'id': 7, 'name': '7'},
 {'id': 8, 'name': '8'},
 {'id': 9, 'name': '9'},
 {'id': 10, 'name': '10'}]

In [84]:
sowing_id = Sowing.objects.filter(farmer_id=2645).order_by('id')[0].id
sowing_id

13

In [85]:
sowing_id = Sowing.objects.get(farmer_id=2645).id
sowing_id

13

In [6]:
procurement_values = Procurement.objects.filter(procurement_group__season_id=get_active_season_id()).values_list('id', 'procurement_group__produce_net_weight',
                                                                  'procurement_group__agent__first_name',
                                                                  'procurement_group__agent__last_name',
                                                                  'procurement_group__procurement_date',
                                                                  'ticket_number', 'vehicle_number','str_weight', 'str_weight_unit',
                                                                  'gunnybag_weight', 'moisture', 'other_deduction', 'reason_for_weight_loss')

procurment_column = ['procurment_id', 'produce_net_weight', 'agent_first_name', 'agent_last_name',
                    'procurement_date', 'ticket_number', 'vehicle_number','str_weight', 'str_weight_unit', 'gunnybag_weight',
                    'moisture', 'other_deduction', 'reason_for_weight_loss']

procurement_df = pd.DataFrame(list(procurement_values),columns=procurment_column)
df

NameError: name 'pd' is not defined

In [2]:
procurement_df['loss'] = ((procurement_df['str_weight'] - procurement_df['produce_net_weight']) / procurement_df['str_weight']) * 100
procurement_df['loss'] = procurement_df['loss'].astype(float).round(2)
print(procurement_df)
procurement_df= procurement_df.drop(['procurment_id'], axis=1)
procurement_df

NameError: name 'procurement_df' is not defined

In [1]:
procurement_df = procurement_df[['agent_first_name','agent_last_name', 'procurement_date', 'ticket_number', 'vehicle_number','produce_net_weight', 'str_weight', 'str_weight_unit',
    'gunnybag_weight', 'moisture', 'other_deduction', 'loss', 'reason_for_weight_loss']]

procurement_df[['produce_net_weight', 'str_weight', 'str_weight_unit','gunnybag_weight', 'moisture', 'other_deduction', 'loss']] = procurement_df[['produce_net_weight', 'str_weight', 'str_weight_unit','gunnybag_weight', 'moisture', 'other_deduction', 'loss']].astype(float)
procurement_df = procurement_df.rename(columns={'agent_first_name' : 'Agent First Name','agent_last_name' : 'Agent Last Name', 'procurement_date' : 'Procurement Date', 'ticket_number' : 'Ticket Number', 'vehicle_number': 'Vehicle Number','produce_net_weight' : 'Produce Net Weight', 'str_weight' : 'Str Weight', 'str_weight_unit' : 'Str Weight Unit',
'gunnybag_weight' : 'Gunnybag Weight', 'moisture' : 'Moisture', 'other_deduction' : 'Other Deduction', 'loss' : 'Loss', 'reason_for_weight_loss': 'Reason for Weight Loss',})
procurement_df


NameError: name 'procurement_df' is not defined

In [158]:
data = {}
data['data'] = procurement_df.to_dict('records')
print(data)
procurement_df.index += 1
total = procurement_df.sum(numeric_only=True)
total.name = 'Total'
procurement_df = procurement_df.append(total.transpose())

{'data': [{'produce_net_weight': Decimal('2389.00'), 'agent_first_name': 'C Mahesha', 'agent_last_name': 'Chikkarajappa', 'procurement_date': datetime.date(2022, 5, 15), 'ticket_number': '2022051501001', 'vehicle_number': 'TN36BW4888', 'str_weight': Decimal('2389.00'), 'str_weight_unit': 1.0, 'gunnybag_weight': Decimal('10.60'), 'moisture': Decimal('0.00'), 'other_deduction': Decimal('0.00'), 'reason_for_weight_loss': None, 'loss': 0.0}, {'produce_net_weight': Decimal('1711.00'), 'agent_first_name': 'C Mahesha', 'agent_last_name': 'Chikkarajappa', 'procurement_date': datetime.date(2022, 5, 16), 'ticket_number': '2022051601001', 'vehicle_number': 'TN36BW4888', 'str_weight': Decimal('1711.00'), 'str_weight_unit': 1.0, 'gunnybag_weight': Decimal('9.00'), 'moisture': Decimal('0.00'), 'other_deduction': Decimal('0.00'), 'reason_for_weight_loss': None, 'loss': 0.0}, {'produce_net_weight': Decimal('1653.00'), 'agent_first_name': 'Muruga_ Sargur', 'agent_last_name': 'Srinivasa', 'procurement_d

TypeError: can only concatenate str (not "int") to str

In [29]:
def get_active_season_id():
    season_obj = Season.objects.get(is_active=True)
    return season_obj.id

In [28]:
def generate_agent_code(user_type_id, season_id, cluster_id):
    print(UserProfileCodeBank.objects.get(season_id=season_id,user_type_id=user_type_id).last_count)
    print(UserProfileCodeBank.objects.get(season_id=season_id,user_type_id=user_type_id).prefix_code)
    print(Cluster.objects.get(id=cluster_id).name[:1])
    prefix = UserProfileCodeBank.objects.get(season_id=season_id,user_type_id=user_type_id).prefix_code
    if user_type_id == 6:
        prefix = prefix + str(Cluster.objects.get(id=cluster_id).name[:1])
    print('prefix: ', prefix)
    last_count =UserProfileCodeBank.objects.get(season_id=season_id,user_type_id=user_type_id).last_count
    generated_count = last_count + 1
    suffix = str(generated_count).zfill(3)
    code = str(prefix) + suffix
    UserProfileCodeBank.objects.filter(season_id=season_id,user_type_id=user_type_id).update(last_count=generated_count)
    print(code)
    return code


In [53]:
firstname="Naveen"
lastname="selvarasu"
mobilenumber=9994443888
agentid=None
email="naveen@gamil.com"

In [54]:
from django.contrib.auth.hashers import make_password
from rest_framework.response import Response
from rest_framework import status


def save_agent():
    season_id = get_active_season_id()
    data = {}
    # new user
    if agentid == None:
        if User.objects.filter(username=mobilenumber).exists():
            data["message"] = "This Agent already Register"
            print(data)
            return Response(data, status=status.HTTP_400_BAD_REQUEST)
        else:
            # create user
            user_obj = User(
                first_name=firstname,
                last_name=lastname,
                username=mobilenumber,
                password=make_password("1234"),
            )
            if email != None:
                user_obj.email = email
            user_obj.save()
            print("user created success")
            # create user profile
            season_id = 4
            user_profile = UserProfile(
                business_id=1,
                user=user_obj,
                code=generate_agent_code(6,season_id,15 ),
                user_type_id=6,
                gender_id=1,
                blood_group_id=2,
                dob="2000-12-12",
                mobile=mobilenumber,
                alternate_mobile="994433400",
                emergency_number="995996494",
                agreement_number="15",
                driving_licence_number="151651452",
                state_id=1,
                district_id=11,
                taluk_id=62,
                address="kallakkkurichi",
                pincode=606221,
                hobli_id=11,
                village_id=33,
                educational_qualification_id=5,
                institution_name="kct",
                university_name="kct",
                aadhaar_number="51441561645845",
                pan_number="bjnsx516151",
                prior_experience_in_other_company=True,
                prior_experience_company_name="ST",
                prior_experience_duration=2,
                date_of_joining="2002-11-20",
                created_by_id=99,
                modified_by_id=99,
            )
           
            user_profile.save()
            print("user profile saved")

            # # assign supervisor
            # if UserHierarchyMap.objects.filter(superior_id=request.data['supervisor_id']).exists():
            #     user_hierarchy_obj = UserHierarchyMap.objects.get(superior_id=request.data['supervisor_id'])
            #     user_hierarchy_obj.subordinate.add(user_obj)
            #     user_hierarchy_obj.save()
            # else:
            #     user_hierarchy_obj = UserHierarchyMap(superior_id=request.data['supervisor_id'],superior_user_type_id=5, subordinate_user_type_id=6)
            #     user_hierarchy_obj.save()
            #     user_hierarchy_obj.subordinate.add(user_obj)
            #     user_hierarchy_obj.save()

            AgentSupervisorSeasonMap.objects.create(
                supervisor_id=30,
                agent_id=user_obj.id,
                season_id=4
            )
            print('saved the supervisor map')
            for cluster_id in [15]:

                if cluster_id != None:
                    if not UserClusterMap.objects.filter(cluster_id=cluster_id, user=user_obj, season_id=season_id).exists():
                        user_cluster_obj = UserClusterMap(
                            cluster_id=cluster_id,
                            user=user_obj,
                            season_id=get_active_season_id(),
                            unique_code='',
                            modified_by_id=99,
                        )
                        user_cluster_obj.save()

            
            AgentMergeloadEnable.objects.create(season_id=season_id, agent_id=user_obj.id)

            #agentwallet
            AgentWallet.objects.create(modified_by_id=99, agent_id=user_obj.id)
            
            data["message"] = "agent saved successfully"
            print("agent save success")
            print(data)
        return Response(data=data, status=status.HTTP_200_OK)

    # old user update
    elif agentid != None:
        print(request.data)
        user_id = agentid
        User.objects.filter(id=user_id).update(
            first_name=firstname,
            last_name=lastname,
            email=email,
        )

        UserProfile.objects.filter(user_id=user_id).update(
            gender_id=1,
            dob="2000-10-20",
            mobile=mobilenumber,
            alternate_mobile="9944334400",
            emergency_number="9959961494",
            agreement_number="15",
            driving_licence_number="151651452",
            state_id=1,
            district_id=11,
            taluk_id=62,
            address="kallakkkurichi",
            pincode=606221,
            hobli_id=11,
            village_id=33,
            educational_qualification_id=5,
            institution_name="kct",
            university_name="kct",
            aadhaar_number="51441561645845",
            pan_number="bjnsx516151",
            prior_experience_in_other_company=True,
            prior_experience_company_name="ST",
            prior_experience_duration=2,
            date_of_joining="2002-11-15",
            created_by=99,
            modified_by=99,
        )
        
        user_profile.save()

        UserClusterMap.objects.filter(user_id=user_id, season_id=season_id).exclude(cluster_id__in=[15]).delete()

        user_obj = User.objects.get(id=user_id)
        # update supervisor
        AgentSupervisorSeasonMap.objects.filter(agent_id=user_id, season_id=season_id).update(
            supervisor_id=request.data['supervisor_id']
        )

        # if not UserHierarchyMap.objects.filter(superior_id=request.data['supervisor_id'], subordinate=user_obj).exists():
        #     if UserHierarchyMap.objects.filter(subordinate=user_obj).exists():
        #         user_hierarchy_obj = UserHierarchyMap.objects.get(subordinate=user_obj)
        #         user_hierarchy_obj.subordinate.remove(user_obj)
        #         user_hierarchy_obj.save()
        #     # assign supervisor
        #     if UserHierarchyMap.objects.filter(superior_id=request.data['supervisor_id'], ).exists():
        #         user_hierarchy_obj = UserHierarchyMap.objects.get(superior_id=request.data['supervisor_id'])
        #         user_hierarchy_obj.subordinate.add(user_obj)
        #         user_hierarchy_obj.save()
        #     else:
        #         user_hierarchy_obj = UserHierarchyMap(superior_id=request.data['supervisor_id'], superior_user_type_id=5, subordinate_user_type_id=6)
        #         user_hierarchy_obj.save()
        #         user_hierarchy_obj.subordinate.add(user_obj)
        #         user_hierarchy_obj.save()


        for cluster_id in [15]:
            if cluster_id == None:
                print("this value is none")
                UserClusterMap.objects.filter(user_id=user_id, season_id=season_id).delete()
            else:
                if not UserClusterMap.objects.filter(
                        cluster_id=cluster_id, user_id=user_id, season_id=season_id
                ).exists():
                    user_cluster_obj = UserClusterMap(
                        cluster_id=cluster_id,
                        user_id=user_id,
                        season_id=4,
                        unique_code='',
                        modified_by=99,
                    )
                    user_cluster_obj.save()

        if "photo" in request.data:
            photo = decode_image(request.data["photo"])
            if UserImage.objects.filter(user_id=user_id).exists():
                UserImage.objects.filter(user_id=user_id).update(
                    image=photo,
                )
            else:
                UserImage.objects.create(
                    image=photo, user_id=user_id, uploaded_by=request.user
                )
        print("employee image updated success")

        data["message"] = "agent updated success"
        return Response(data=data, status=status.HTTP_200_OK)
save_agent()


user created success
48
A
R
prefix:  AR
AR049
user profile saved
saved the supervisor map
agent save success
{'message': 'agent saved successfully'}


<Response status_code=200, "text/html; charset=utf-8">

# akka

In [42]:
far_obj=Farmer.objects.filter().values_list()

for i in far_obj:
    print(i)


# user_obj=list(User.objects.filter().values_list("id"))
# print(user_obj)


(2646, 'Ramkumar', 'Ravi', 'FA2800', 'FA2607', None, 1, 100000, 2, 2, 227, 1, 1, None, 'ragowthaman@gmail.com', '9003832990', None, None, None, '', None, None, None, None, None, None, None, None, None, '', 1, None, False, True, False, False, True, 30, datetime.datetime(2020, 4, 7, 18, 4, 13, 646905, tzinfo=<UTC>), 2, 30, 30, datetime.datetime(2020, 4, 7, 18, 4, 13, 648723, tzinfo=<UTC>), datetime.datetime(2021, 5, 26, 6, 45, 21, 60060, tzinfo=<UTC>))
(2663, 'Kulti02', 'Farmer', 'FA2904', 'FA2617', None, 1, 616161, 2, 2, 227, 1, 1, None, None, '8111010654', None, None, None, '', None, None, None, None, None, None, None, None, None, '', 1, None, False, True, False, False, True, 23, datetime.datetime(2020, 4, 10, 14, 54, 4, 571540, tzinfo=<UTC>), 2, 23, 23, datetime.datetime(2020, 4, 10, 14, 54, 4, 573620, tzinfo=<UTC>), datetime.datetime(2021, 5, 26, 6, 45, 21, 57203, tzinfo=<UTC>))
(2643, 'Arun', 'Arun', 'FA2797', 'FA2605', None, 1, 154649, 2, 2, 227, 1, 1, None, 'ragowthaman@gmail.com'

(12362, 'Shivarudrappa', 'Rajappa', '22CGE037', 'FB2704', None, 1, 571111, 1, 11, 62, 14, 65, None, None, '7090050439', None, None, '237551245714', '', None, None, None, None, None, None, None, None, None, '', 1, None, False, True, False, False, True, 23, datetime.datetime(2022, 4, 30, 9, 51, 55, 427364, tzinfo=<UTC>), 2, 23, 80, datetime.datetime(2022, 4, 30, 9, 51, 55, 428178, tzinfo=<UTC>), datetime.datetime(2022, 7, 12, 12, 14, 18, 907140, tzinfo=<UTC>))
(12370, 'G M Guruswamappa', 'Mallappa', '22CGE045', 'FB2712', None, 1, 571111, 1, 11, 62, 14, 65, None, None, '1222255885', None, None, '726467410158', '', None, None, None, None, None, None, None, None, None, '', 1, None, False, True, False, False, True, 23, datetime.datetime(2022, 4, 30, 10, 3, 53, 577841, tzinfo=<UTC>), 2, 23, 80, datetime.datetime(2022, 4, 30, 10, 3, 53, 578588, tzinfo=<UTC>), datetime.datetime(2022, 6, 18, 5, 24, 20, 812963, tzinfo=<UTC>))
(12371, 'Siddarajappa', 'Siddalingappa', '22CGE047', 'FB2713', None, 1,

(1867, 'Puttanaika', 'Sannanaika', '1866', 'FA1866', 1, 1, None, 1, 26, 171, 23, 254, None, None, None, None, 1, None, '', None, None, None, None, None, None, None, None, None, '', 6, None, False, True, False, False, False, None, None, 1, 1, 1, datetime.datetime(2020, 3, 20, 5, 40, 27, 105480, tzinfo=<UTC>), datetime.datetime(2021, 5, 26, 6, 45, 30, 576251, tzinfo=<UTC>))
(1638, 'Puttaraju', 'Sannagandanaika', '19NPB057', 'FA1637', 1, 1, None, 1, 26, 171, 23, 254, None, None, None, None, 1, None, '', None, None, None, None, None, None, None, None, None, '', 6, None, False, True, False, False, False, None, None, 1, 1, 1, datetime.datetime(2020, 3, 20, 5, 40, 16, 130393, tzinfo=<UTC>), datetime.datetime(2021, 5, 26, 6, 45, 30, 548216, tzinfo=<UTC>))
(1872, 'Rathnamma', 'Dasegowda', '1871', 'FA1871', 1, 1, None, 1, 26, 171, 23, 254, None, None, None, None, 1, None, '', None, None, None, None, None, None, None, None, None, '', 6, None, False, True, False, False, False, None, None, 1, 1, 1,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [41]:
import pandas as pd
  
# creating and initializing a list
values= [['Vansh',455,],['Radhika',250],['Deepak',495],
         ['Soni',400],['Radhika',350],['Vansh',450]]
 
# creating a pandas dataframe
df = pd.DataFrame(values,columns=['Name','Total_Marks'])
 
# Applying lambda function to find
# percentage of 'Total_Marks' column
# using df.assign()
df = df.assign(Percentage = lambda Mrk: (sum(Mrk['Total_Marks'])))
 
# displaying the data frame
df

,Name,Total_Marks,Percentage
0,Vansh,455,2400
1,Radhika,250,2400
2,Deepak,495,2400
3,Soni,400,2400
4,Radhika,350,2400
5,Vansh,450,2400
